In [3]:
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO

# locate standings of league
standings_url = "https://www.sports-reference.com/cfb/conferences/mwc/2023.html"
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

# store each link to schedule of each team
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/schools/' in l]
team_urls = [f"https://www.sports-reference.com{l.replace('/2023', '/2023-schedule')}" for l in links]
all_matches = [];

for team_url in team_urls:
    team_name = team_url.split("/")[5].replace("-", " ")

    data = requests.get(team_url)
    html_content = StringIO(data.text)

    # list of possible table names for pandas to look for
    table_names = ["14 Games Table", "13 Games Table", "12 Games Table"]

    for table_name in table_names:
        try:
            matches = pd.read_html(html_content, match=table_name)[0]
            break
        except ValueError:
            continue

    # convert date format for indexing
    matches['Date'] = pd.to_datetime(matches['Date'], format='%b %d, %Y')
    matches['Date'] = matches['Date'].dt.strftime('%Y-%m-%d')

    # get links of each gamelog of each team
    soup = BeautifulSoup(data.text)
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if l and '/2023/gamelog/' in l]

    data = requests.get(f"https://www.sports-reference.com{links[0]}")
    html_content = StringIO(data.text)

    # find offensive stats table & format
    try:
        game_logs = pd.read_html(html_content, match="Offensive Game Log Table")[0]
    except ValueError:
        print(team_name + " didn't find game log")
        continue

    game_logs.columns = game_logs.columns.droplevel()

    # making columns with duplicate names have unique identifier
    columns = game_logs.columns
    counts = {}
    def rename_column(col):
        if col in counts:
            counts[col] += 1
            return f"{counts[col]}{col}"
        else:
            counts[col] = 0
            return col
    new_columns = [rename_column(col) for col in columns]
    game_logs.columns = new_columns
    game_logs = game_logs.rename(columns={"Pct" : "Pct_Passing" ,"Yds" : "Yds_Passing", "TD" : "TD_Passing", "1Yds" : "Yds_Rushing", "Avg" : "Avg_Rushing", "1TD" : "TD_Rushing", "2Yds" : "Yds_Tot_Off", "1Avg" : "Avg_Tot_Off.", "Tot" : "Tot_First_Downs", "No." : "No_Penalties", "1Tot" : "Tot_Turnovers"})

    # merging data with each match based on date we formatted
    try:
        team_data = matches.merge(game_logs[["Date", "Pct_Passing", "Yds_Passing", "TD_Passing", "Yds_Rushing", "Avg_Rushing", "TD_Rushing", "Yds_Tot_Off", "Avg_Tot_Off.", "Tot_First_Downs", "No_Penalties", "Tot_Turnovers"]], on = "Date")
        team_data = team_data.rename(columns = {"Unnamed: 5" : "Home/Away", "Unnamed: 8" : "W/L"})
    except ValueError:
        print("error merging")
        continue

    # adding column for team name
    team_data["Team"] = team_name
    # adding to match list
    all_matches.append(team_data)
    # timer to make sure website doesnt time out
    time.sleep(1)

# creating dataframe and creating csv file
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("matches.csv")